# 評価指標と目的関数

・モデルの予測値の評価に用いるのが評価指標であり、目的関数はモデルが学習中に最適化される関数である。

・上手く学習を進めるには、目的関数が微分可能である必要がある。

## カスタム評価指標とカスタム目的関数
・モデルやライブラリによっては、提供されていない評価指標や目的関数をユーザーが定義できる。

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

# データセットロード
cancer = load_breast_cancer()
X = cancer.data; y = cancer.target
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=42)

# xgboostのデータ構造へ変換
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(valid_x, label=valid_y)


# カスタム評価指標: logloss(xgboostの'binary:logistic'と等価)
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0 - preds)
    return grad, hess

# カスタム評価指標: error
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom-error', float(sum(labels != (preds > 0.0))) / len(labels)

# パラメータ
params = {'random_state': 42}
num_round = 50
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

bst = xgb.train(params, dtrain, num_round, watchlist, obj=logregobj, feval=evalerror, verbose_eval=0)

# 確率値に変換
pred_val = bst.predict(dvalid)
pred = 1.0 / (1.0 + np.exp(-pred_val))
logloss = log_loss(valid_y, pred)
print('logloss: ', logloss)

logloss:  0.09399718567695808


# 評価指標の最適化

## 評価指標最適化のアプローチ

例として一部抜粋

・評価指標がRMSEやloglossならば、モデルの目的関数も同じものを指定できる。RMSLEならば学習データの目的変数を対数変換し、目的関数をRMSEとして学習させた後、指数関数で変換をもとに戻した予測値を提出する。

・異なる評価指標を選択し、それを評価対象としてセットしたEarlyStoppingを用いて、最適になる時点で学習を止める方法。

## 閾値の最適化

予測確率ではなく、正例か負例のラベルを提出する評価指標においては、通常は予測確率からある閾値で分けて、正例か負例に振る。しかしF1-scoreの場
合は正例の割合や正しく予測出来ている割合によって、F1-scoreを最大化する閾値が異なるため、その閾値を求める必要がある。方法として、

・0.01~0.99間を0.01刻みで、すべてのスコアを走査し、最良を探す。

・最適化アルゴリズムを使う。

    ・Nelder-Mead法: 最適化対象となる目的関数が微分可能でなくても使用できる。
    
    ・COBYLA法: 制約式(制約条件)を設定できる。
    
    ・SLSQP法: 最適化対象となる目的関数、制約式が微分可能であることを前提とする。
    
    等。Nelder-MeadやCOBYLAは比較的安定した解が求められる。

In [2]:
from sklearn.metrics import f1_score
from scipy.optimize import minimize

rand = np.random.RandomState(seed=42)
# 0~1まで1万刻みでprob生成
train_y_prob = np.linspace(0, 1, 10000)

# train_y_prob.size分のブール値を真の値として、生成ベクトルを標準化したものを予測値として、それぞれ定義
train_y = pd.Series(rand.uniform(0.0, 1.0, train_y_prob.size) < train_y_prob)
train_pred_prob = np.clip(train_y_prob * np.exp(rand.standard_normal(train_y_prob.shape) * 0.3), 0.0, 1.0)

init_threshold = 0.5
init_score = f1_score(train_y, train_pred_prob >= init_threshold)
print(f'init_threshold: {init_threshold} init_score: {init_score}')


# 目的関数
def f1_opt(x):
    return -f1_score(train_y, train_pred_prob >= x)


# Nelder-Mead法を選択、resultにはベストな閾値が返される
result = minimize(f1_opt, x0=np.array([0.5]), method='Nelder-Mead')
print('\nNelder-Mead method result:\n%s\n' % result)
best_threshold = result['x'].item()
best_score = f1_score(train_y, train_pred_prob >= best_threshold)
print(f'best_threshold: {best_threshold} best_score: {best_score}\n')

# COBYLA法を選択
result = minimize(f1_opt, x0=np.array([0.5]), method='COBYLA', options={'maxiter': 10000, 'catol': 0.8})
print('\nCOBYLA method result:\n%s\n' % result)
best_threshold = result['x'].item()
best_score = f1_score(train_y, train_pred_prob >= best_threshold)
print(f'best_threshold: {best_threshold} best_score: {best_score}')

init_threshold: 0.5 init_score: 0.7221549636803875

Nelder-Mead method result:
 final_simplex: (array([[0.35585937],
       [0.35576172]]), array([-0.76296101, -0.76296101]))
           fun: -0.7629610069536134
       message: 'Optimization terminated successfully.'
          nfev: 31
           nit: 14
        status: 0
       success: True
             x: array([0.35585937])

best_threshold: 0.35585937499999987 best_score: 0.7629610069536134


COBYLA method result:
     fun: -0.7629701400510878
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 22
  status: 1
 success: True
       x: array([0.35644531])

best_threshold: 0.3564453125 best_score: 0.7629701400510878


## 閾値の最適化をoofで行うべきか？

### oof (out-of-fold)

交差検証において、分割したfoldの内1つは変数を与えずに予測用（答えを知らない）とし、他を学習用とする。

これを、fold数分1つずつずらしながら、すべてのfoldの予測値を作成する。

予測値はモデルの評価やスタッキングの特徴量に用いる。

閾値の最適化でoofを用いることによって、閾値やスコアのぶれを考慮した最適化が行える。

In [3]:
from sklearn.model_selection import KFold

rand = np.random.RandomState(seed=42)
# 0~1まで1万刻みでprob生成
train_y_prob = np.linspace(0, 1, 10000)

# train_y_prob.size分のブール値を真の値として、生成ベクトルを標準化したものを予測値として、それぞれ定義
train_y = pd.Series(rand.uniform(0.0, 1.0, train_y_prob.size) < train_y_prob)
train_pred_prob = np.clip(train_y_prob * np.exp(rand.standard_normal(train_y_prob.shape) * 0.3), 0.0, 1.0)

thresholds = []
scores_tr = []
scores_va = []

kf = KFold(n_splits=4, random_state=42, shuffle=True)
for i, (tr_idx, va_idx) in enumerate(kf.split(train_pred_prob)):
    tr_pred_prob, va_pred_prob = train_pred_prob[tr_idx], train_pred_prob[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # 目的関数
    def f1_opt(x):
        return -f1_score(train_y, train_pred_prob >= x)
    
    result = minimize(f1_opt, x0=np.array([0.5]), method='Nelder-Mead')
    threshold = result['x'].item()
    score_tr = f1_score(tr_y, tr_pred_prob >= threshold)
    score_va = f1_score(va_y, va_pred_prob >= threshold)
    print('Fold', i)
    print(f'threshold: {threshold}, score_tr: {score_tr}, score_va: {score_va}')
    
    thresholds.append(threshold)
    scores_tr.append(score_tr)
    scores_va.append(score_va)

    
threshold_test = np.mean(thresholds)
print('\nthresholds mean: ', threshold_test)

Fold 0
threshold: 0.35585937499999987, score_tr: 0.7621823268827232, score_va: 0.7653611210923463
Fold 1
threshold: 0.35585937499999987, score_tr: 0.7627950455713953, score_va: 0.7634677131644666
Fold 2
threshold: 0.35585937499999987, score_tr: 0.7621525724423418, score_va: 0.7653131452167928
Fold 3
threshold: 0.35585937499999987, score_tr: 0.7647197362223269, score_va: 0.7577553154409201

thresholds mean:  0.35585937499999987


## 予測確率とその調整

分類タスクにおいて評価指標の最適化を考えた場合、評価指標が目的関数となっていれば妥当な予測確率を期待できるが、そうでない場合は予測確率に歪みが生じる場合がある。AUCを通した場合は大小関係さえあっていれば問題ないが、loglossの場合は予測確率がずれているとペナルティが生じる。

一般的に予測確率が歪むケースとして、

・データが十分でない

・モデルのアルゴリズム上、妥当な予測確率だすよう最適化されていない場合

RandomForestは目的関数をloglossとせず、GBDT、NN、RogisticRegression等とは異なるアルゴリズムで分類をするため、確率は歪んでいる。

こういった場合、事後に予測確率を調整することによって、スコアが改善する場合がある。

### 予測確率の調整
・予測値をn乗する：　確率を十分に学習できていないと考え、補正として予測値をn乗(0.9~1.1)する後処理を加える。

・極端に0や1に近い確率のクリップ：　評価指標にペナルティがある場合、それを避けるために出力する確率をクリップする方法

・スタッキング：　スタッキングの2層目のモデルとして、GBDT、NN、RogisticRegression等の妥当な確率を出力するモデルを選定する方法。

・CalibratedClassifierCV：　scikit-learnモジュールのCalibratedClassifierCVを用いて、予測確率を補正（較正）する方法。

In [4]:
## 予測値をn乗する

import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# データセットロード
cancer = load_breast_cancer()
X = cancer.data; y = cancer.target
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=42)

lr_model = LogisticRegression(random_state=42)
rf_model = RandomForestClassifier(random_state=42)

lr_model.fit(train_x, train_y)
rf_model.fit(train_x, train_y)

lr_pred = lr_model.predict_proba(valid_x)[:, 1]
rf_pred = rf_model.predict_proba(valid_x)[:, 1]

print('LogisticRegression logloss score: ', log_loss(valid_y, lr_pred))
print('RandomForest logloss score: ', log_loss(valid_y, rf_pred))
print('RandomForest logloss score with post-processing: ', log_loss(valid_y, rf_pred**1.1))

LogisticRegression logloss score:  0.07837392262879811
RandomForest logloss score:  0.09544270629552579
RandomForest logloss score with post-processing:  0.0949390543490502


In [5]:
## CalibratedClassifierCV
## 交差検証同様にデータセットを分割し、訓練と補正をそれぞれ行う

from sklearn.calibration import CalibratedClassifierCV

rf_model = RandomForestClassifier(random_state=42)
# シグモイド関数による補正を選択
# 事前学習済みモデルを使う場合は、method='prefit'を指定する
calib_rf = CalibratedClassifierCV(rf_model, method='sigmoid', cv=2)
calib_rf.fit(train_x, train_y)

calib_rf_pred = calib_rf.predict_proba(valid_x)[:, 1]
print('Calibrated RandomForest logloss score: ', log_loss(valid_y, calib_rf_pred))

Calibrated RandomForest logloss score:  0.08199515752044927


In [6]:
# 補正前、補正後のRandomForest予測確率の統計量
pd.DataFrame(
    [pd.Series(rf_pred.ravel()).describe().transpose(), pd.Series(calib_rf_pred.ravel()).describe().transpose()], 
    index=['RF pred stats', 'Calibrated RF pred stats']
)

,count,mean,std,min,25%,50%,75%,max
RF pred stats,114.0,0.619912,0.442806,0.00000,0.020000,0.920000,0.990000,1.000000
Calibrated RF pred stats,114.0,0.628026,0.444807,0.01586,0.023299,0.971116,0.984013,0.985087
